Code for Code Explaination


In [ ]:
import requests
import json
import os


OPENAI_API_KEY = "OPENAI_API_KEY"
DEEPSEEK_API_KEY = "DEEPSEEK_API_KEY"

GEMINI_API_KEY = "GEMINI_API_KEY"

# API Endpoints
OPENAI_ENDPOINT = "https://api.openai.com/v1/chat/completions"
DEEPSEEK_ENDPOINT = "https://api.deepseek.com/chat/completions"
GEMINI_ENDPOINT = "https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash:generateContent"

# --- Helper Functions for LLM Interactions ---

def get_openai_response(prompt_text: str, code_content: str, api_key: str) -> str:

    if not api_key or api_key == "OPENAI_API_KEY":
        return "OpenAI API Key not configured. Please set OPENAI_API_KEY."

    messages = [
        {"role": "user", "content": prompt_text},
        {"role": "user", "content": f"Here is the code file content:\n```\n{code_content}\n```"}
    ]

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }
    payload = {
        "model": "gpt-4o",
        "messages": messages,
        "max_tokens": 1000
    }

    try:
        response = requests.post(OPENAI_ENDPOINT, headers=headers, json=payload)
        response.raise_for_status() # Raise an HTTPError for bad responses (4xx or 5xx)
        data = response.json()
        return data['choices'][0]['message']['content'].strip()
    except requests.exceptions.RequestException as e:
        return f"Error connecting to OpenAI API: {e}"
    except KeyError:
        return f"Unexpected response format from OpenAI API: {response.text}"

def get_deepseek_response(prompt_text: str, code_content: str, api_key: str) -> str:
    """
    Gets a response from Deepseek's chat completions model.
    Assumes Deepseek API is similar to OpenAI's chat completions.
    """
    if not api_key or api_key == "YOUR_DEEPSEEK_API_KEY":
        return "Deepseek API Key not configured. Please set DEEPSEEK_API_KEY."

    messages = [
        {"role": "user", "content": prompt_text},
        {"role": "user", "content": f"Here is the code file content:\n```\n{code_content}\n```"}
    ]

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }
    payload = {
        "model": "deepseek-chat",
        "messages": messages,
        "max_tokens": 1000
    }

    try:
        response = requests.post(DEEPSEEK_ENDPOINT, headers=headers, json=payload)
        response.raise_for_status()
        data = response.json()
        return data['choices'][0]['message']['content'].strip()
    except requests.exceptions.RequestException as e:
        return f"Error connecting to Deepseek API: {e}"
    except KeyError:
        return f"Unexpected response format from Deepseek API: {response.text}"

def get_gemini_response(prompt_text: str, code_content: str, api_key: str) -> str:

    chat_history = []
    chat_history.append({"role": "user", "parts": [{"text": prompt_text}]})
    chat_history.append({"role": "user", "parts": [{"text": f"Here is the code file content:\n```\n{code_content}\n```"}]})

    payload = {"contents": chat_history}
    api_url = f"{GEMINI_ENDPOINT}?key={api_key}"

    headers = {
        'Content-Type': 'application/json'
    }

    try:
        response = requests.post(api_url, headers=headers, data=json.dumps(payload))
        response.raise_for_status()
        result = response.json()

        if result.get('candidates') and result['candidates'][0].get('content') and \
           result['candidates'][0]['content'].get('parts') and \
           result['candidates'][0]['content']['parts'][0].get('text'):
            return result['candidates'][0]['content']['parts'][0]['text'].strip()
        else:
            return f"Unexpected response format from Gemini API: {response.text}"
    except requests.exceptions.RequestException as e:
        return f"Error connecting to Gemini API: {e}"
    except Exception as e:
        return f"An unexpected error occurred with Gemini API: {e}"

# --- Main Logic ---

def main():
    print("Welcome to the LLM Code Analyzer!")
    print("-" * 30)

    user_prompt = input(" `Explain the code. First in 40-50 words explain what this code file does in the context of the project repository. Then, explain what this code file does(write it in 100-120 words and write 1 small line describing each method in the code.)`;\n")

    code_file_path = input("Enter the path to your code file : ")

    code_content = ""
    try:
        with open(code_file_path, 'r', encoding='utf-8') as f:
            code_content = f.read()
        print(f"\nSuccessfully loaded code from '{code_file_path}' (first 100 chars):")
        print(code_content[:100] + "..." if len(code_content) > 100 else code_content)
    except FileNotFoundError:
        print(f"Error: File not found at '{code_file_path}'. Please check the path.")
        return
    except Exception as e:
        print(f"Error reading file: {e}")
        return

    print("\nGetting responses from LLMs...\n")

    # Get response from GPT-4o
    print("--- GPT-4o Response ---")
    gpt_response = get_openai_response(user_prompt, code_content, OPENAI_API_KEY)
    print(gpt_response)
    print("-" * 30)

    # Get response from Deepseek
    print("\n--- Deepseek Response ---")
    deepseek_response = get_deepseek_response(user_prompt, code_content, DEEPSEEK_API_KEY)
    print(deepseek_response)
    print("-" * 30)

    # Get response from Gemini
    print("\n--- Gemini Response ---")
    gemini_response = get_gemini_response(user_prompt, code_content, GEMINI_API_KEY)
    print(gemini_response)
    print("-" * 30)


if __name__ == "__main__":
    main()

Code for code refactoring


In [ ]:
import requests
import json
import os


OPENAI_API_KEY = "OPENAI_API_KEY"
DEEPSEEK_API_KEY = "DEEPSEEK_API_KEY"
GEMINI_API_KEY = "GEMINI_API_KEY"

# API Endpoints
OPENAI_ENDPOINT = "https://api.openai.com/v1/chat/completions"
DEEPSEEK_ENDPOINT = "https://api.deepseek.com/chat/completions"
GEMINI_ENDPOINT = "https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash:generateContent"

# --- Helper Functions for LLM Interactions ---

def get_openai_response(prompt_text: str, code_content: str, api_key: str) -> str:

    if not api_key or api_key == "OPENAI_API_KEY":
        return "OpenAI API Key not configured. Please set OPENAI_API_KEY."

    messages = [
        {"role": "user", "content": prompt_text},
        {"role": "user", "content": f"Here is the code file content:\n```\n{code_content}\n```"}
    ]

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }
    payload = {
        "model": "gpt-4o",
        "messages": messages,
        "max_tokens": 1000
    }

    try:
        response = requests.post(OPENAI_ENDPOINT, headers=headers, json=payload)
        response.raise_for_status() # Raise an HTTPError for bad responses (4xx or 5xx)
        data = response.json()
        return data['choices'][0]['message']['content'].strip()
    except requests.exceptions.RequestException as e:
        return f"Error connecting to OpenAI API: {e}"
    except KeyError:
        return f"Unexpected response format from OpenAI API: {response.text}"

def get_deepseek_response(prompt_text: str, code_content: str, api_key: str) -> str:
    """
    Gets a response from Deepseek's chat completions model.
    Assumes Deepseek API is similar to OpenAI's chat completions.
    """
    if not api_key or api_key == "YOUR_DEEPSEEK_API_KEY":
        return "Deepseek API Key not configured. Please set DEEPSEEK_API_KEY."

    messages = [
        {"role": "user", "content": prompt_text},
        {"role": "user", "content": f"Here is the code file content:\n```\n{code_content}\n```"}
    ]

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }
    payload = {
        "model": "deepseek-chat",
        "messages": messages,
        "max_tokens": 1000
    }

    try:
        response = requests.post(DEEPSEEK_ENDPOINT, headers=headers, json=payload)
        response.raise_for_status()
        data = response.json()
        return data['choices'][0]['message']['content'].strip()
    except requests.exceptions.RequestException as e:
        return f"Error connecting to Deepseek API: {e}"
    except KeyError:
        return f"Unexpected response format from Deepseek API: {response.text}"

def get_gemini_response(prompt_text: str, code_content: str, api_key: str) -> str:

    chat_history = []
    chat_history.append({"role": "user", "parts": [{"text": prompt_text}]})
    chat_history.append({"role": "user", "parts": [{"text": f"Here is the code file content:\n```\n{code_content}\n```"}]})

    payload = {"contents": chat_history}
    api_url = f"{GEMINI_ENDPOINT}?key={api_key}"

    headers = {
        'Content-Type': 'application/json'
    }

    try:
        response = requests.post(api_url, headers=headers, data=json.dumps(payload))
        response.raise_for_status()
        result = response.json()

        if result.get('candidates') and result['candidates'][0].get('content') and \
           result['candidates'][0]['content'].get('parts') and \
           result['candidates'][0]['content']['parts'][0].get('text'):
            return result['candidates'][0]['content']['parts'][0]['text'].strip()
        else:
            return f"Unexpected response format from Gemini API: {response.text}"
    except requests.exceptions.RequestException as e:
        return f"Error connecting to Gemini API: {e}"
    except Exception as e:
        return f"An unexpected error occurred with Gemini API: {e}"

# --- Main Logic ---

def main():
    print("Welcome to the LLM Code Analyzer!")
    print("-" * 30)

    user_prompt = input(" `Refactor the following code file and provide the complete, cleaned, and refactored version as output. Include brief comments (12–15 words) next to each section where refactoring was performed, explaining the changes made. Do not include any additional explanations or descriptions, just give the refactored code as output`;\n")

    code_file_path = input("Enter the path to your code file : ")

    code_content = ""
    try:
        with open(code_file_path, 'r', encoding='utf-8') as f:
            code_content = f.read()
        print(f"\nSuccessfully loaded code from '{code_file_path}' (first 100 chars):")
        print(code_content[:100] + "..." if len(code_content) > 100 else code_content)
    except FileNotFoundError:
        print(f"Error: File not found at '{code_file_path}'. Please check the path.")
        return
    except Exception as e:
        print(f"Error reading file: {e}")
        return

    print("\nGetting responses from LLMs...\n")

    # Get response from GPT-4o
    print("--- GPT-4o Response ---")
    gpt_response = get_openai_response(user_prompt, code_content, OPENAI_API_KEY)
    print(gpt_response)
    print("-" * 30)

    # Get response from Deepseek
    print("\n--- Deepseek Response ---")
    deepseek_response = get_deepseek_response(user_prompt, code_content, DEEPSEEK_API_KEY)
    print(deepseek_response)
    print("-" * 30)

    # Get response from Gemini
    print("\n--- Gemini Response ---")
    gemini_response = get_gemini_response(user_prompt, code_content, GEMINI_API_KEY)
    print(gemini_response)
    print("-" * 30)


if __name__ == "__main__":
    main()

Code for Code Quality Metrics

In [ ]:
import requests
import json
import os


OPENAI_API_KEY = "OPENAI_API_KEY"
DEEPSEEK_API_KEY = "DEEPSEEK_API_KEY"
GEMINI_API_KEY = "GEMINI_API_KEY"

# API Endpoints
OPENAI_ENDPOINT = "https://api.openai.com/v1/chat/completions"
DEEPSEEK_ENDPOINT = "https://api.deepseek.com/chat/completions"
GEMINI_ENDPOINT = "https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash:generateContent"

# --- Helper Functions for LLM Interactions ---

def get_openai_response(prompt_text: str, code_content: str, api_key: str) -> str:

    if not api_key or api_key == "OPENAI_API_KEY":
        return "OpenAI API Key not configured. Please set OPENAI_API_KEY."

    messages = [
        {"role": "user", "content": prompt_text},
        {"role": "user", "content": f"Here is the code file content:\n```\n{code_content}\n```"}
    ]

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }
    payload = {
        "model": "gpt-4o",
        "messages": messages,
        "max_tokens": 1000
    }

    try:
        response = requests.post(OPENAI_ENDPOINT, headers=headers, json=payload)
        response.raise_for_status() # Raise an HTTPError for bad responses (4xx or 5xx)
        data = response.json()
        return data['choices'][0]['message']['content'].strip()
    except requests.exceptions.RequestException as e:
        return f"Error connecting to OpenAI API: {e}"
    except KeyError:
        return f"Unexpected response format from OpenAI API: {response.text}"

def get_deepseek_response(prompt_text: str, code_content: str, api_key: str) -> str:
    """
    Gets a response from Deepseek's chat completions model.
    Assumes Deepseek API is similar to OpenAI's chat completions.
    """
    if not api_key or api_key == "YOUR_DEEPSEEK_API_KEY":
        return "Deepseek API Key not configured. Please set DEEPSEEK_API_KEY."

    messages = [
        {"role": "user", "content": prompt_text},
        {"role": "user", "content": f"Here is the code file content:\n```\n{code_content}\n```"}
    ]

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }
    payload = {
        "model": "deepseek-chat",
        "messages": messages,
        "max_tokens": 1000
    }

    try:
        response = requests.post(DEEPSEEK_ENDPOINT, headers=headers, json=payload)
        response.raise_for_status()
        data = response.json()
        return data['choices'][0]['message']['content'].strip()
    except requests.exceptions.RequestException as e:
        return f"Error connecting to Deepseek API: {e}"
    except KeyError:
        return f"Unexpected response format from Deepseek API: {response.text}"

def get_gemini_response(prompt_text: str, code_content: str, api_key: str) -> str:

    chat_history = []
    chat_history.append({"role": "user", "parts": [{"text": prompt_text}]})
    chat_history.append({"role": "user", "parts": [{"text": f"Here is the code file content:\n```\n{code_content}\n```"}]})

    payload = {"contents": chat_history}
    api_url = f"{GEMINI_ENDPOINT}?key={api_key}"

    headers = {
        'Content-Type': 'application/json'
    }

    try:
        response = requests.post(api_url, headers=headers, data=json.dumps(payload))
        response.raise_for_status()
        result = response.json()

        if result.get('candidates') and result['candidates'][0].get('content') and \
           result['candidates'][0]['content'].get('parts') and \
           result['candidates'][0]['content']['parts'][0].get('text'):
            return result['candidates'][0]['content']['parts'][0]['text'].strip()
        else:
            return f"Unexpected response format from Gemini API: {response.text}"
    except requests.exceptions.RequestException as e:
        return f"Error connecting to Gemini API: {e}"
    except Exception as e:
        return f"An unexpected error occurred with Gemini API: {e}"

# --- Main Logic ---

def main():
    print("Welcome to the LLM Code Analyzer!")
    print("-" * 30)

    user_prompt = input(" `Calculate the cyclomatic complexity, maintainability index and vulnerability category that suit most from CVE categories for the following code. Just check out the codes and from the code try to answer . just write the numbers.In your response only give the detected values of these attributes. Don't give any explanation`;\n")

    code_file_path = input("Enter the path to your code file : ")

    code_content = ""
    try:
        with open(code_file_path, 'r', encoding='utf-8') as f:
            code_content = f.read()
        print(f"\nSuccessfully loaded code from '{code_file_path}' (first 100 chars):")
        print(code_content[:100] + "..." if len(code_content) > 100 else code_content)
    except FileNotFoundError:
        print(f"Error: File not found at '{code_file_path}'. Please check the path.")
        return
    except Exception as e:
        print(f"Error reading file: {e}")
        return

    print("\nGetting responses from LLMs...\n")

    # Get response from GPT-4o
    print("--- GPT-4o Response ---")
    gpt_response = get_openai_response(user_prompt, code_content, OPENAI_API_KEY)
    print(gpt_response)
    print("-" * 30)

    # Get response from Deepseek
    print("\n--- Deepseek Response ---")
    deepseek_response = get_deepseek_response(user_prompt, code_content, DEEPSEEK_API_KEY)
    print(deepseek_response)
    print("-" * 30)

    # Get response from Gemini
    print("\n--- Gemini Response ---")
    gemini_response = get_gemini_response(user_prompt, code_content, GEMINI_API_KEY)
    print(gemini_response)
    print("-" * 30)


if __name__ == "__main__":
    main()